In [121]:
import okx.MarketData as MarketData
import csv
import json
from ultils import *
import pandas as pd
import ta

In [122]:
flag = "0"  # Production trading:0 , demo trading:1
marketDataAPI =  MarketData.MarketAPI(flag=flag)

### input of API is here

In [123]:
# From date and to date
before = "2023-01-01 00:00:00.0" # from
after = "2024-01-01 00:00:00.0" # to 
id = "BTC-USDT" # kind of coin
bar = '4H' # kind of bar 1m, 3m, 5m, 15m, 30m, 1H, 2H, 4H


In [124]:
# Convert bar to timestamp
def convert_bar_to_timestamp(bar):
    bar_to_timestamp = {
        '1m': 1,
        '3m': 3,
        '5m': 5,
        '15m': 15,
        '30m': 30,
        '1H': 60,
        '2H': 120,
        '4H': 240
    }
    
    return bar_to_timestamp.get(bar, "Invalid bar")


timestamp = convert_bar_to_timestamp(bar)

In [125]:
# Convert to miliseconds
before = datetime_to_ms(before)
after = datetime_to_ms(after)
end = after

# Retrieve history candlestick charts from recent years
def get_candlesticks(instId, start_ms, end_ms, bar):
    try:
        result = marketDataAPI.get_history_candlesticks(
            instId=instId,
            after=(start_ms),
            before=(end_ms),
            bar=bar,
        )
        return result
    except Exception as e:
        print(f"Error: {e}")
        return None


### run API and save to .json

In [126]:
# Get history candlesticks
# Schedule to run the script every 10 times per second
result_all = []
while after - before >= 5940000*timestamp:
    result = get_candlesticks(id, after, before, bar)
    result_all.append(result)
    after = after - 5940000*timestamp - 60000*timestamp

result = get_candlesticks(id, after, before, bar)
result_all.append(result)



header:  {'Content-Type': 'application/json', 'x-simulated-trading': '0'}
domain: https://www.okx.com
url: /api/v5/market/history-candles?instId=BTC-USDT&after=1704042000000&before=1672506000000&bar=4H
header:  {'Content-Type': 'application/json', 'x-simulated-trading': '0'}
domain: https://www.okx.com
url: /api/v5/market/history-candles?instId=BTC-USDT&after=1702602000000&before=1672506000000&bar=4H
header:  {'Content-Type': 'application/json', 'x-simulated-trading': '0'}
domain: https://www.okx.com
url: /api/v5/market/history-candles?instId=BTC-USDT&after=1701162000000&before=1672506000000&bar=4H
header:  {'Content-Type': 'application/json', 'x-simulated-trading': '0'}
domain: https://www.okx.com
url: /api/v5/market/history-candles?instId=BTC-USDT&after=1699722000000&before=1672506000000&bar=4H
header:  {'Content-Type': 'application/json', 'x-simulated-trading': '0'}
domain: https://www.okx.com
url: /api/v5/market/history-candles?instId=BTC-USDT&after=1698282000000&before=16725060000

In [127]:
# Convert data to DataFrame
rows = []
try:
    for item in result_all:
        for entry in item['data']:
            row = {
                'ts': entry[0],
                'o': entry[1],
                'h': entry[2],
                'l': entry[3],
                'c': entry[4],
                'vol': entry[5],
                'volCcy': entry[6],
                'volCcyQuote': entry[7],
                'confirm': entry[8]
            }
            rows.append(row)

    df = pd.DataFrame(rows)
    df['ts'] = pd.to_datetime(df['ts'].astype('int64'), unit='ms')
    df[['o', 'h', 'l', 'c', 'vol', 'volCcy', 'volCcyQuote']] = df[['o', 'h', 'l', 'c', 'vol', 'volCcy', 'volCcyQuote']].astype('float64')
except Exception as e:
    print(f"Error: {e}")

In [128]:
df_indi = ta.add_all_ta_features(df, "o", "h", "l", "c", "vol", fillna=True)
df_indi.dropna(inplace=True)

In [129]:
df_indi

,ts,o,h,l,c,vol,volCcy,volCcyQuote,confirm,volume_adi,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
0,2023-12-31 16:00:00,42457.6,42720.0,42430.1,42612.1,726.903768,3.097006e+07,3.097006e+07,1,185.800515,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,42612.100000,0.000000,0.000000,0.000000
1,2023-12-31 12:00:00,42522.0,42658.5,42346.0,42457.5,1337.709328,5.685362e+07,5.685362e+07,1,-197.319437,...,-0.028950,-0.005790,-0.023160,6.310343,1.262069,5.048275,42545.663642,-0.362808,-0.363467,-0.362808
2,2023-12-31 08:00:00,42539.7,42888.0,42369.5,42522.0,2341.122437,9.980496e+07,9.980496e+07,1,-1161.311028,...,-0.039229,-0.012478,-0.026751,18.729758,4.755607,13.974151,42535.452966,0.151917,0.151801,-0.211442
3,2023-12-31 04:00:00,42174.1,42678.9,42153.0,42539.5,1302.770196,5.521440e+07,5.521440e+07,1,-549.189873,...,-0.043558,-0.018694,-0.024865,18.915809,7.587647,11.328162,42537.205087,0.041155,0.041147,-0.170374
4,2023-12-31 00:00:00,42147.5,42432.2,41971.6,42174.0,1279.389075,5.394538e+07,5.394538e+07,1,-704.183165,...,-0.114993,-0.037954,-0.077039,18.593218,9.788761,8.804457,42387.584104,-0.859201,-0.862914,-1.028112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2185,2023-01-01 12:00:00,16558.5,16574.0,16537.2,16560.2,134.338404,2.224023e+06,2.224023e+06,1,143979.435248,...,-0.543433,-0.555752,0.012319,-16.202852,-9.770105,-6.432748,16709.971332,-0.267996,-0.268356,-61.137330
2186,2023-01-01 08:00:00,16531.1,16558.5,16510.0,16558.4,140.791604,2.327889e+06,2.327889e+06,1,144119.646268,...,-0.557847,-0.556171,-0.001676,-20.557096,-11.927503,-8.629593,16688.554923,-0.010869,-0.010870,-61.141554
2187,2023-01-01 04:00:00,16538.6,16552.0,16502.0,16531.1,277.635612,4.588001e+06,4.588001e+06,1,144165.178508,...,-0.575953,-0.560128,-0.015826,-21.393741,-13.820751,-7.572990,16665.306523,-0.164871,-0.165007,-61.205620
2188,2023-01-01 00:00:00,16546.1,16562.8,16512.7,16538.5,360.815116,5.969795e+06,5.969795e+06,1,144175.981356,...,-0.580125,-0.564127,-0.015998,-20.184279,-15.093456,-5.090823,16616.250402,0.044764,0.044754,-61.188254
